In [2]:
from gurobipy import *

M = Model()
# number of players in the team
n = 14
# player strength list, either 2.5 or 3.0 for each player
N = [2.5, 3.0, 3.0, 2.5, 3.0, 2.5, 3.0, 3.0, 2.5, 2.5, 2.5, 3.0, 2.5, 2.5]
# opponent strength list, either 2.5 or 3.0 for each court position
N0 = [3.0, 3.0, 3.0, 3.0, 3.0, 2.5, 3.0, 3.0]
# number of games in the tournament
m = 8
# number of slots for a player to play in per game
o = 8
# minimum number of games each player must play
p = 3
# availability matrix. 1 if the ith player can make the jth game, 0 if they cannot
A = [[1 for j in range(m)] for i in range(n)] 
# preference matrix. 1 if the ith player prefers the jth game and kth slot, -1 if they prefer not to, 0 if no preference
P = [[[0 for k in range(o)] for j in range(m)] for i in range(n)] 
for j in range(0,m):
    P[1][j][6] = 1 #test preferences - TODO: add actual player preferences
    P[2][j][7] = 1

# 1 if a player i plays in game j on the kth court, 0 if they do not
x = M.addVars(n, m, o, vtype=GRB.BINARY)
# 0 if the kth court for the jth game is balanced or favors the Romanian team, 1 if it disfavors the Romanian team
bal = M.addVars(m,5, vtype=GRB.BINARY)
M.update()

for i in range(0,n):
    M.addConstrs(x.sum(i, j, '*') <= A[i][j] for j in range(0,m)) #players can only play one court in a match, 
                                                                  #or none if they are not available
    M.addConstr(x.sum(i, '*', '*'), GRB.GREATER_EQUAL, p) #players must play at least p games in the season

for j in range(0,m):
    for k in [0,2,4]:
        expr = LinExpr(-1*N0[k]-N0[k+1])
        for i in range(0,n):
            expr.add(x[i,j,k], N[i])
            expr.add(x[i,j,k+1], N[i])
        expr.add(bal[j, int((k+1)/2)], .5)
        M.addConstr(expr >= 0) #ensures that bal[j,k] for the doubles courts will be 0 if the romanian team is at least as strong
    for k in [6,7]:
        expr = LinExpr(-1*N0[k])
        for i in range(0,n):
            expr.add(x[i,j,k], N[i])
        expr.add(bal[j, k-3], .5)
        M.addConstr(expr >= 0) #ensures that bal[j,k] for the singles courts will be 0 if the romanian team is at least as strong

for k in range(0,o):
    M.addConstrs(x.sum('*', j, k) == 1 for j in range(0,m)) #all courts must have exactly one player
    
M.update()

# consecutive games stuff
inds = M.addVars(n,m, vtype=GRB.BINARY)
for i in range(0,n):
    for j in range(0,m-1):
        M.addConstr(x.sum(i, j, '*') + x.sum(i,j+1, '*') - inds[i,j] <= 1)

expr = LinExpr(-188)
opponentSum = 0;
for k in range(0,o):
    for j in range(0,m):
        opponentSum += N0[j]
        for i in range(0,5):
            expr.add(bal[j,i], -1) # attempt to balance as many matches as possible
            expr.addConstant(1)
        for i in range(0,n):
            expr.add(x[i,j,k], N[i]) # "overall team strength" - TODO: rework condition
            expr.add(x[i,j,k], P[i][j][k]) # preferences
            expr.add(inds[i,j], -0.1) #consecutive games penalty

M.setObjective(expr, GRB.MAXIMIZE)

M.optimize()


Optimize a model with 328 rows, 1048 columns and 5290 nonzeros
Variable types: 0 continuous, 1048 integer (1048 binary)
Coefficient statistics:
  Matrix range     [5e-01, 3e+00]
  Objective range  [8e-01, 8e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective 94.3000000
Presolve removed 0 rows and 14 columns
Presolve time: 0.01s
Presolved: 328 rows, 1034 columns, 4778 nonzeros
Variable types: 0 continuous, 1034 integer (1034 binary)

Root relaxation: objective 1.792000e+02, 694 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     179.2000000  179.20000  0.00%     -    0s

Explored 0 nodes (694 simplex iterations) in 0.06 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 179.2 94.3 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.7920000000

In [9]:
for j in range(0,m):
    for k in range(0,5):
        print(bal[j,k].x)
    print("NEWLINE")

1.0
1.0
0.0
0.0
0.0
NEWLINE
0.0
1.0
1.0
0.0
0.0
NEWLINE
1.0
1.0
0.0
0.0
0.0
NEWLINE
1.0
1.0
0.0
0.0
0.0
NEWLINE
1.0
1.0
0.0
0.0
0.0
NEWLINE
1.0
0.0
1.0
0.0
0.0
NEWLINE
1.0
0.0
1.0
0.0
0.0
NEWLINE
1.0
0.0
1.0
0.0
0.0
NEWLINE
